In [253]:
import pandas as pd

# Load the dataframe
df = pd.read_excel("C:\\Users\\logans1\\Desktop\\Excel_GroupBy\\Leases - Legal update.xlsx", sheet_name="Sheet1")

# Create a list of rows to drop
rows_to_drop = df[df['Lease Type'].isin(['RE - Wind', 'RE - Solar', 'Right of Way'])].index

# Drop the rows
df.drop(rows_to_drop, inplace=True)

# Create abbreviations for merge with Lease Number
df['Lease Type abbv'] = df['Lease Type'].apply(lambda x: 'AG-' if x == 'Agriculture' else 'COMM-' if x == 'Commercial' else 'TOW-' if x == 'Tower' else 'ES-' if x == 'Ecosystem Services' else 'SM-' if x == 'Solid Mineral' else 'RAP-' if x == 'Road Access Permit' else 'OG-' if x == 'Oil and Gas' else 'REC-' if x == 'Recreation' else 'TAP-' if x == 'Temporary Access Permit' else 'NSO-' if x == 'No Surface Occupancy' else 'AGRMT-' if x == 'Agreement' else 'EP-' if x == 'Exploration Permit' else 'GCS-' if x == 'Geologic Carbon Storage' else 'TAP-' if x == 'Temporary Access Permit Consideration' else 'GRND-' if x == 'Ground' else x[:3])

# use the str function to convert the integer values to strings before concatenating them
df['Transaction Number'] = df.apply(lambda row: row['Lease Type abbv'] + str(row['Lease Number']), axis=1)

df


,Internal ID,Lease Type,Lease Number,Item,Related Asset,Acreage,Legal Description,Lease Type abbv,Transaction Number
0,8601,Agriculture,100001,Grazing Rent,FAM002093,70.00,E2,AG-,AG-100001
1,8601,Agriculture,100001,Grazing Rent,FAM002094,640.00,ALL,AG-,AG-100001
2,8601,Agriculture,100001,Dry Crop Land,FAM002093,250.00,E2,AG-,AG-100001
3,8704,Agriculture,100014,Grazing Rent Bid,FAM004437,637.12,"LTS 1-2, S2NE, SE, W2",AG-,AG-100014
4,8704,Agriculture,100014,Grazing Rent Bid,FAM004635,609.18,"LTS 1-4, S2N2, S2",AG-,AG-100014
...,...,...,...,...,...,...,...,...,...
46318,1221454,Agriculture,115146,Grazing Rent,FAM006397,640.00,ALL,AG-,AG-115146
46319,1221457,Agriculture,115147,Non-Billable Acres,FAM004207,3.00,ALL,AG-,AG-115147
46320,1221457,Agriculture,115147,Grazing Rent,FAM004207,352.40,ALL,AG-,AG-115147
46321,1221457,Agriculture,115147,Dry Crop Land Fixed,FAM004207,94.60,ALL,AG-,AG-115147


In [254]:
# Check if the columns "Internal ID", "Item", and "Related Asset" are present in the dataframe
columns_to_drop = ["Internal ID", "Item", "Related Asset", "Lease Type abbv"]
columns_present = [column in df.columns for column in columns_to_drop]

# Only drop the columns that are present in the dataframe
if any(columns_present):
    df = df.drop(columns=columns_to_drop)

df

,Lease Type,Lease Number,Acreage,Legal Description,Transaction Number
0,Agriculture,100001,70.00,E2,AG-100001
1,Agriculture,100001,640.00,ALL,AG-100001
2,Agriculture,100001,250.00,E2,AG-100001
3,Agriculture,100014,637.12,"LTS 1-2, S2NE, SE, W2",AG-100014
4,Agriculture,100014,609.18,"LTS 1-4, S2N2, S2",AG-100014
...,...,...,...,...,...
46318,Agriculture,115146,640.00,ALL,AG-115146
46319,Agriculture,115147,3.00,ALL,AG-115147
46320,Agriculture,115147,352.40,ALL,AG-115147
46321,Agriculture,115147,94.60,ALL,AG-115147


In [255]:
# Group the dataframe by the Lease Number column and sum the values in the Acreage column
summary_df = df.groupby('Transaction Number')['Acreage'].sum().reset_index()

# Rename the Acreage column to Total Acreage
summary_df = summary_df.rename(columns={'Acreage': 'Total Lease Acreage'})

# The summary dataframe now contains the total acreage for each Lease Number
print(summary_df)

     Transaction Number  Total Lease Acreage
0             AG-100001              960.000
1             AG-100014            47859.790
2             AG-100112             2660.550
3             AG-100145             1280.000
4             AG-100148             5175.020
...                 ...                  ...
4198         TOW-112838               40.000
4199         TOW-113440                0.028
4200         TOW-113960              160.000
4201         TOW-114153                0.370
4202         TOW-114318                2.760

[4203 rows x 2 columns]


In [256]:
df_legals = df.groupby('Transaction Number')['Legal Description'].apply(lambda x: ', '.join([str(i) for i in x])).reset_index()
df_legals.columns = ['Transaction Number', 'Lease Legals']

df_legals

,Transaction Number,Lease Legals
0,AG-100001,"E2, ALL, E2"
1,AG-100014,"LTS 1-2, S2NE, SE, W2, LTS 1-4, S2N2, S2, LTS ..."
2,AG-100112,"ALL, LTS 11,12; S2SE, S2SW, SW, SWSE, W2NW, S2..."
3,AG-100145,"ALL, ALL, ALL, ALL"
4,AG-100148,"LOTS 2-4, S2NW, NWSW; ALL PARTS OF LOT 1, S2NE..."
...,...,...
4198,TOW-112838,"nan, ALL"
4199,TOW-113440,SESWSE
4200,TOW-113960,NW
4201,TOW-114153,NENESE


In [257]:
# Merge df_legals and summary_df into one df
merged_df = pd.merge(df_legals, summary_df, on='Transaction Number')
merged_df

,Transaction Number,Lease Legals,Total Lease Acreage
0,AG-100001,"E2, ALL, E2",960.000
1,AG-100014,"LTS 1-2, S2NE, SE, W2, LTS 1-4, S2N2, S2, LTS ...",47859.790
2,AG-100112,"ALL, LTS 11,12; S2SE, S2SW, SW, SWSE, W2NW, S2...",2660.550
3,AG-100145,"ALL, ALL, ALL, ALL",1280.000
4,AG-100148,"LOTS 2-4, S2NW, NWSW; ALL PARTS OF LOT 1, S2NE...",5175.020
...,...,...,...
4198,TOW-112838,"nan, ALL",40.000
4199,TOW-113440,SESWSE,0.028
4200,TOW-113960,NW,160.000
4201,TOW-114153,NENESE,0.370


In [258]:
# Save the dataframe to a new Excel file
merged_df.to_excel('Lease Update.xlsx', index=False)